# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.141/12437/1 Dashboard: http://10.20.0.141:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1037,Michael,0.5262715250857068,-0.9004752736897077
2000-01-01 00:00:01,1007,Tim,0.4667278409880242,0.2500411448595712
2000-01-01 00:00:02,1001,Hannah,-0.6766770264492958,0.7339380753033906
2000-01-01 00:00:03,1046,Tim,-0.8834290432511718,0.5526177174720361
2000-01-01 00:00:04,971,Sarah,-0.8062900691663797,-0.5284814474747053
2000-01-01 00:00:05,1006,Dan,-0.7108400325161068,0.25548633551355726
2000-01-01 00:00:06,993,Zelda,-0.36657521263321713,0.2768433724829218
2000-01-01 00:00:07,1041,Laura,0.6403002858884601,-0.22344620123815373
2000-01-01 00:00:08,969,Frank,0.9912647821304894,0.17945938026115682


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1032,Norbert,0.6428737400009792,0.4553748660450039
2000-01-30 00:00:01,990,Tim,0.5888852762949455,0.8820763040622082
2000-01-30 00:00:02,999,Hannah,-0.26485903782379205,-0.39770438529850094
2000-01-30 00:00:03,1011,Tim,-0.9637274366716804,-0.33312551822238934
2000-01-30 00:00:04,967,Zelda,-0.024050136746512907,0.07214316396927734
2000-01-30 00:00:05,984,Yvonne,-0.34141249269985274,-0.8384499213558798
2000-01-30 00:00:06,1006,Dan,-0.3164106265720228,-0.9920981278086087
2000-01-30 00:00:07,1022,Ray,-0.7644012396491,0.15700302424283907
2000-01-30 00:00:08,940,Zelda,0.2797825804829961,0.7746504632849454


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1037,Michael,0.526272,-0.900475
1,2000-01-01 00:00:01,1007,Tim,0.466728,0.250041
2,2000-01-01 00:00:02,1001,Hannah,-0.676677,0.733938
3,2000-01-01 00:00:03,1046,Tim,-0.883429,0.552618
4,2000-01-01 00:00:04,971,Sarah,-0.806290,-0.528481


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1037,Michael,0.526272,-0.900475
1,2000-01-01 00:00:01,1007,Tim,0.466728,0.250041
2,2000-01-01 00:00:02,1001,Hannah,-0.676677,0.733938
3,2000-01-01 00:00:03,1046,Tim,-0.883429,0.552618
4,2000-01-01 00:00:04,971,Sarah,-0.806290,-0.528481


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 5.63 s, sys: 728 ms, total: 6.36 s
Wall time: 4.26 s


name
Alice       0.002180
Bob         0.002571
Charlie     0.001229
Dan        -0.001284
Edith       0.001109
Frank      -0.000442
George     -0.000119
Hannah     -0.000851
Ingrid      0.001037
Jerry       0.001625
Kevin      -0.002044
Laura      -0.000177
Michael    -0.000810
Norbert     0.003575
Oliver     -0.002087
Patricia   -0.001242
Quinn       0.002829
Ray         0.001511
Sarah       0.000824
Tim         0.003272
Ursula      0.000389
Victor     -0.001346
Wendy      -0.000624
Xavier     -0.004087
Yvonne     -0.002410
Zelda      -0.000085
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.85 s, sys: 188 ms, total: 2.04 s
Wall time: 1.4 s


name
Alice       0.002180
Bob         0.002571
Charlie     0.001229
Dan        -0.001284
Edith       0.001109
Frank      -0.000442
George     -0.000119
Hannah     -0.000851
Ingrid      0.001037
Jerry       0.001625
Kevin      -0.002044
Laura      -0.000177
Michael    -0.000810
Norbert     0.003575
Oliver     -0.002087
Patricia   -0.001242
Quinn       0.002829
Ray         0.001511
Sarah       0.000824
Tim         0.003272
Ursula      0.000389
Victor     -0.001346
Wendy      -0.000624
Xavier     -0.004087
Yvonne     -0.002410
Zelda      -0.000085
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.52 s, sys: 141 ms, total: 1.67 s
Wall time: 1.19 s


name
Alice       0.002180
Bob         0.002571
Charlie     0.001229
Dan        -0.001284
Edith       0.001109
Frank      -0.000442
George     -0.000119
Hannah     -0.000851
Ingrid      0.001037
Jerry       0.001625
Kevin      -0.002044
Laura      -0.000177
Michael    -0.000810
Norbert     0.003575
Oliver     -0.002087
Patricia   -0.001242
Quinn       0.002829
Ray         0.001511
Sarah       0.000824
Tim         0.003272
Ursula      0.000389
Victor     -0.001346
Wendy      -0.000624
Xavier     -0.004087
Yvonne     -0.002410
Zelda      -0.000085
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html